In [2]:
import requests
import bs4

print(requests.__version__)
print(bs4.__version__)

2.31.0
4.12.3


## 네이버 상태 코드


In [3]:
URL = 'https://www.naver.com/'

req = requests.get(URL)
print(req.status_code)

200


In [5]:
#req.text

In [9]:
from bs4 import BeautifulSoup 
import pandas as pd 

with open('index.html', 'r', encoding='UTF8') as f:

    # step 01 : 데이터 수집
    contents = f.read()

    # step 02 : 데이터 파싱 (순수한 HTML 파일을 BeautifulSoup 객체로 변환)
    # 농구공(=HTML, 문자열) ===> 규칙 (농구장)
    # 농구공 ===> 축구장(크롤링)으로 던짐 (농구공이 축구공으로 변하는 매직) BeautifulSoup() 클래스를 통해서 축구공으로 변함
    # 축구(=BeautifulSoup 클래스 문법)규칙 을 따르면 됨
    
    soup = BeautifulSoup(contents, 'lxml')
    # print(soup)
    
#     print(soup.h2)
#     print(soup.ul)
#     print("-----")
#    print(soup.ul.li)
    # 4개의 li 태그에 있는 회사명을 모두 가져오는 것이 목적

    # step 03 : 데이터 수집 위한 특정 태그 찾기
    companies = []
    print(soup.find_all('li'))

    # step 04 : 데이터 가공
    for tag in soup.find_all('li'):
        companies.append(tag.text)
    print(companies)

    # step 05 : 처리된 데이터 저장 pandas 데이터프레임
    crawling_dict = {'회사명': companies}
    result = pd.DataFrame(crawling_dict)
    print(result)

    # step 06 : csv 파일로 내보내기 or DB로 내보내기
    result.to_csv("result.csv", index=False)



[<li>애플</li>, <li>삼성</li>, <li>노키아</li>, <li>LG</li>]
['애플', '삼성', '노키아', 'LG']
   회사명
0   애플
1   삼성
2  노키아
3   LG


In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd  

custom_header = {
	'referer' : 'https://music.bugs.co.kr/',
	'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}


url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
req = requests.get(url, headers = custom_header)

# 페이지의 HTML을 BeautifulSoup 객체로 변환
soup = BeautifulSoup(req.content, 'html.parser') #text, tbody

# 노래 제목 추출

song = []
for title in soup.find_all('p',class_ ='title'):
    song.append(title.text)
#print(song)

# 데이터프레임 생성
df = pd.DataFrame({'곡': song})

# 결과 출력
print(df)



                                  곡
0                 \nLove wins all\n
1                          \nWife\n
2                         \nTo. X\n
3                      \nLove 119\n
4                          \n에피소드\n
..                              ...
95  \n머물러주오 (Prod. 안신애 & Philtre)\n
96                       \n운이 좋았지\n
97                      \nSnowman\n
98                \nCool With You\n
99         \n사랑은 먼 길을 돌아온 메아리 같아서\n

[100 rows x 1 columns]


In [20]:
#강사님 코드
import requests
from bs4 import BeautifulSoup
import pandas as pd 

def crawling(soup) :
    # print(soup)
    tbody = soup.find("tbody")  #이부분 다시공부
    result = []
    for p in tbody.find_all('p', class_ = 'title'):
        result.append(p.get_text().strip())
    return result

def main() :
    custom_header = {
        'referer' : 'https://music.bugs.co.kr/',
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    url = "https://music.bugs.co.kr/chart" # 크롤링 하려는 웹사이트
    req = requests.get(url, headers = custom_header)
    
    soup = BeautifulSoup(req.text, "html.parser")
    crawling(soup)

    titles = crawling(soup)
    print(pd.DataFrame({"노래제목" : titles}))


if __name__ == "__main__" :
    main()

                              노래제목
0                    Love wins all
1                             Wife
2                         Love 119
3                            To. X
4                             에피소드
..                             ...
95                         사랑을 하다가
96  Smoke (Prod. Dynamicduo, Padi)
97                        미워 (Ego)
98                           밤, 바다
99            사랑은 먼 길을 돌아온 메아리 같아서

[100 rows x 1 columns]


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random
warnings.filterwarnings('ignore')

company_code = '005930' # 삼성전자
url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    
headers = { 
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }

N = 12
df =None

for pageNum in range(1, N+1):
    url=f'https://finance.naver.com/item/sise_day.naver?code=005930&page={pageNum}'
 
    req = requests.get(url, headers=headers)
    soup  = BeautifulSoup(req.content,"html.parser")
    result = pd.read_html(req.text, encoding='euc-kr')[0] #리스트안에 판다스데이터프레임에 들어간 형태 type도 리스트로 나옴
    #print(result)
    df =pd.concat([df, result],ignore_index=True)
df


,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024.01.31,72700.0,1600.0,73400.0,74000.0,72500.0,14874547.0
2,2024.01.30,74300.0,100.0,75000.0,75300.0,73700.0,12244418.0
3,2024.01.29,74400.0,1000.0,73800.0,75200.0,73500.0,13976521.0
4,2024.01.26,73400.0,700.0,73700.0,74500.0,73300.0,11160062.0
...,...,...,...,...,...,...,...
175,2023.08.09,68900.0,1300.0,68000.0,69600.0,67900.0,17259673.0
176,2023.08.08,67600.0,900.0,69000.0,69100.0,67400.0,14664709.0
177,2023.08.07,68500.0,200.0,67700.0,69200.0,67600.0,10968505.0
178,2023.08.04,68300.0,500.0,68800.0,69100.0,68200.0,12360193.0


In [32]:

req = requests.get(url, headers=headers)


/var/folders/dk/vy49vgx92_3d76t7s3b_x5c40000gn/T/ipykernel_97776/4192256325.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  pd.read_html(req.text, encoding='euc-kr')


[            날짜       종가     전일비       시가       고가       저가         거래량
 0          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 1   2024.01.31  72700.0  1600.0  73400.0  74000.0  72600.0  12023736.0
 2   2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
 3   2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
 4   2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
 5   2024.01.25  74100.0   100.0  74200.0  74800.0  73700.0  11737747.0
 6          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 7          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 8          NaN      NaN     NaN      NaN      NaN      NaN         NaN
 9   2024.01.24  74000.0  1200.0  75200.0  75200.0  73500.0  12860661.0
 10  2024.01.23  75200.0   100.0  75700.0  75800.0  74300.0  14786224.0
 11  2024.01.22  75100.0   400.0  75900.0  76000.0  75000.0  19673375.0
 12  2024.01.19  74700.0  3000.0  73500.0  74700.0  73000.0  233

In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import warnings
import time
import random
warnings.filterwarnings('ignore') 

def crawling(url, headers, soup):
    #각 페이지에 맞춰서 모든 페이지를 가지고 오도록하는 설정
    last_page = int(soup.select_one('td.pgRR').a['href'].split('=')[-1])
    
    df = None
    count = 0
    for page in range(1, last_page + 1):
      req = requests.get(f'{url}&page={page}', headers=headers)
      df = pd.concat([df, pd.read_html(req.text, encoding = "euc-kr")[0]], ignore_index=True)
      if count > 10:
        break
      count += 1
      time.sleep( random.uniform(2,4)) 

    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df

def main():
    company_code = '005930' # 삼성전자
    url ="https://finance.naver.com/item/sise_day.nhn?code=" + company_code
    
    headers = { 
             'referer' : 'https://finance.naver.com/item/sise.naver?code=005930',
             'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    result = crawling(url, headers, soup)
    print(result)

if __name__ == "__main__":
    main()

             날짜       종가     전일비       시가       고가       저가         거래량
0    2024.01.31  72700.0  1600.0  73400.0  74000.0  72500.0  14874547.0
1    2024.01.30  74300.0   100.0  75000.0  75300.0  73700.0  12244418.0
2    2024.01.29  74400.0  1000.0  73800.0  75200.0  73500.0  13976521.0
3    2024.01.26  73400.0   700.0  73700.0  74500.0  73300.0  11160062.0
4    2024.01.25  74100.0   100.0  74200.0  74800.0  73700.0  11737747.0
..          ...      ...     ...      ...      ...      ...         ...
115  2023.08.10  68000.0   900.0  68300.0  68500.0  67800.0  10227311.0
116  2023.08.09  68900.0  1300.0  68000.0  69600.0  67900.0  17259673.0
117  2023.08.08  67600.0   900.0  69000.0  69100.0  67400.0  14664709.0
118  2023.08.07  68500.0   200.0  67700.0  69200.0  67600.0  10968505.0
119  2023.08.04  68300.0   500.0  68800.0  69100.0  68200.0  12360193.0

[120 rows x 7 columns]
